* Additional covariate: Baseline intensity varies by care home size.
* Data is daily number of cases from February to July. 
* 1000 Care homes
* 330 homes had cases, 670 no cases
* 2000 total cases
* 50% homes received discharge transfer from hospital
* 3000 discharges in total
* Data Frame 1: 1 row per day, 1 column per care home ID, entries are numbers of cases
* Data Frame 2: 1 row per day, 1 column per care home ID, entries are numbers of discharges
* Data frame 3: 1 row per covariate, 1 column per care home ID, entries are covariates (to start with just one, care home size, likely as a category with 4 groups).



In [ ]:
MAX_BEDS_PER_HOME = 255
MAX_CATEGORIES = 4

N_CARE_HOMES = 1000
N_CASES = 2000
N_CASE_HOMES = 330
N_DISCHARGES = 3000
N_DISCHARGE_HOMES = 500
N_DAYS = 181
N_COVARIATES = 1

In [3]:
COVARIATES_FILE = "covariates.csv"
CASES_FILE = "cases.csv"
DISCHARGES_FILE = "discharges.csv"

In [17]:
import numpy as np
import scipy.stats

In [13]:
def compactify(array):
    '''Take an array of uint32s and returns it in the smallest datatype.'''
    if array.max() < 256:
        compact_array = array.astype('uint8')
    elif array.max() < 65536:
        compact_array = array.astype('uint16')
    else:
        compact_array = array
    return compact_array


def read_and_tidy_data(filename):
    '''Read in the CSV file in `filename`, sort by the values in the first row
    (i.e. ensure that data are sorted by care homes), then return separately
    the first row (the ID list) and the subsequent data in the smallest data type
    into which it will fit.
    
    Assumes all values are integers smaller than 2**32'''

    read_data = np.loadtxt(filename, dtype=np.uint32, delimiter=',')
    sorted_read_data = read_data[:, read_data[0, :].argsort()]
    care_home_ids = compactify(sorted_read_data[0, :])
    values = compactify(sorted_read_data[1:, :])
    return care_home_ids, values
    

care_home_ids, cases = read_and_tidy_data(CASES_FILE)
covariate_care_home_ids, covariates = read_and_tidy_data(COVARIATES_FILE)
discharge_care_home_ids, discharges = read_and_tidy_data(DISCHARGES_FILE)

In [14]:
assert np.equal(care_home_ids, covariate_care_home_ids).all()
assert np.equal(care_home_ids, discharge_care_home_ids).all()

Let the intensity of cases at care home $i$ on day $t$ be:

$$
    \lambda_i(t) = v_i + r_c \sum_{s<t} f_c (t - s)n_i(s) + r_h \sum_{s<t} f_h(t-s) h_i(s)
$$

where:

* $v_i$ is the baseline intensity, which depends on the size of the care home. Initially, band care home sizes into 4 groups, so $v_i$ becomes four free fit parameters
* $r_c$ is the self-excitation parameter, representing the increased risk of an outbreak following an introduction of a case
* $r_h$ is the excitation by hospital paramteter, representing the increased risk of an outbreak following a hospital transfer
* $s$ and $t$ are integer numbers of days
* $f_c(t)$, $f_h(t)$ are probability distributions

The distribution $f_c(t)$ is approximated by the serial case interval distribution $\mathrm{gamma}(6.5, 0.62)$. In this definition the two parameters to $\mathrm{gamma}$ are the _mean_ $E$ and the _coefficient of variation_ $c_v$. Scipy's definition of the gamma PDF instead takes the _shape_ parameter $k$ and the _scale_ parameter $\theta$.

The mean $E=k\theta$, and the coefficient of variation is $c_v=\sigma/E=1/\sqrt{k}$. Thus to obtain the parameters needed:

$$
\begin{align}
    k &= c_v^{-2} \\
    \theta &= \frac{E}{k} = E c_v^2
\end{align}
$$

In [25]:
# Define parameters for excitation distributions

self_excitation_mean = 6.5
self_excitation_cv = 0.62

discharge_excitation_mean = self_excitation_mean
discharge_excitation_cv = self_excitation_cv

self_excitation_shape = self_excitation_cv ** -2
self_excitation_scale = self_excitation_mean * self_excitation_cv ** 2

discharge_excitation_shape = discharge_excitation_cv ** -2
discharge_excitation_scale = discharge_excitation_mean * discharge_excitation_cv ** 2

In [51]:
def carehome_intensity_null(covariates, cases, baseline_intensities):
    '''
    Returns the intensity of cases at all care homes on all dates, assuming no
    effect due to cases or discharges.
    Shape is (N_DATES, N_CARE_HOMES).
    Requires:
    Input data:
     - covariates: a (1, N_CARE_HOMES) array of care home size bands
     - cases: an (N_DATES, N_CARE_HOMES) array of integers representing
              the number of cases at a given care home on a given day
    Fit parameters:
     - baseline_intensities: a 1d array of baseline intensities as a function of
                             care home size band
    '''
    return np.zeros_like(cases, dtype=np.float64) + baseline_intensities[covariates]


def single_excitation(triggers, shape, scale):
    '''
    Calculates a single set of excitation terms in the form
        e_i(t) = \\sum_{s<t} f(t - s) triggers_i(s)
    where f is the gamma pdf with given shape and scale,
    and triggers is a 2-d array indexed as (t, i)'''

    n_dates, n_care_homes = triggers.shape
    date_range = np.arange(n_dates)[:, np.newaxis]

    full_f_c = scipy.stats.gamma.pdf(n_dates - date_range, a=shape, scale=scale)
    output = np.zeros_like(triggers, dtype=np.float64)

    for s in range(n_dates):
        output[s:] += full_f_c[s:] * triggers[s]
    return output


def carehome_intensity(covariates, cases, baseline_intensities, r_c, discharges=None, r_h=None):
    '''
    Returns the intensity of cases at all care homes on all dates.
    Shape is (N_DATES, N_CARE_HOMES).
    Requires:
    Input data:
     - covariates: a (1, N_CARE_HOMES) array of care home size bands
     - cases: an (N_DATES, N_CARE_HOMES) array of integers representing
              the number of cases at a given care home on a given day
     - discharges: an (N_DATES, N_CARE_HOMES array of integers representing
              the number of discharges from hospital into a given care home
              on a given day
    Fit parameters:
     - baseline_intensities: a 1d array of baseline intensities as a function of
                             care home size band
     - r_c: self-excitation
     - r_h: excitation by hospital. If None, then calculation of this term is skipped
    '''

    output = carehome_intensity_null(covariates, cases, baseline_intensities)
    output += r_c * single_excitation(cases, self_excitation_shape, self_excitation_scale)
    
    if r_h is not None:
        output += r_c * single_excitation(discharges, discharge_excitation_shape, discharge_excitation_scale)
    
    return output

In [50]:
# Arbitrary test data
baseline_intensities = np.asarray([0.5, 1.0, 1.5, 2.0])
r_c = 0.9
r_h = 0.5

In [54]:
%timeit carehome_intensity_null(covariates, cases, baseline_intensities)
%timeit carehome_intensity(covariates, cases, baseline_intensities, r_c)
%timeit carehome_intensity(covariates, cases, baseline_intensities, r_c, discharges, r_h)

253 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
87.5 ms ± 676 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
174 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
def likelihood(intensity, cases):
    '''
    Sum the following over all care homes i: data on days t=1, 2, ..., T
    with cases observed on days t_{i;j}, then use log-likelihood
        \sum_j \ln \lambda_i (t_{i;j}) - \sum_{t=1}^T \lambda_i (t)
    '''

    return np.sum(np.log(intensity[cases > 0])) - np.sum(intensity)

In [60]:
print(likelihood(carehome_intensity_null(covariates, cases, baseline_intensities), cases))
print(likelihood(carehome_intensity(covariates, cases, baseline_intensities, r_c), cases))
print(likelihood(carehome_intensity(covariates, cases, baseline_intensities, r_c, discharges, r_h), cases))

-227664.7330609811
-229371.4374904857
-231970.126054767


In [62]:
%timeit likelihood(carehome_intensity_null(covariates, cases, baseline_intensities), cases)
%timeit likelihood(carehome_intensity(covariates, cases, baseline_intensities, r_c), cases)
%timeit likelihood(carehome_intensity(covariates, cases, baseline_intensities, r_c, discharges, r_h), cases)

655 µs ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
93.1 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
197 ms ± 9.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
